In [1]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc = SparkContext(appName="NetWordCountState")
ssc = StreamingContext(sc, 10)

In [3]:
def update_func(new_val, last_sum):
    return sum(new_val) + (last_sum or 0)

In [4]:
# To maintain status info we need to use checkpoints.
checkpointDir = "file:///tmp/spark" # toUri('/tmp/spark') == "file:///tmp/spark"
ssc.checkpoint(checkpointDir)  # create dir for checkpoint archives

lines = ssc.socketTextStream("localhost", 9876)
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .updateStateByKey(update_func)

counts.pprint()

## Ejecutar desde línea de comandos

```bash 
nc -lk 9999
```
donde se podrán ir introduciendo las palabras


## Iniciar el proceso spark

In [5]:
ssc.start()

-------------------------------------------
Time: 2021-04-24 10:39:00
-------------------------------------------
('A7-KM-30', 1)
('1234UUU', 1)

-------------------------------------------
Time: 2021-04-24 10:39:10
-------------------------------------------
('A7-KM-30', 3)
('1234EEE', 2)
('A7-KM-01', 1)
('A7-KM-15', 1)
('1234UUU', 4)
('1234AAA', 2)
('A7-KM-50', 3)



## Para detener el proceso ejecutad esta celda

In [6]:
ssc.stop()
sc.stop()